In [2]:
#load libraries.  Using a legacy version of Keras because we trained the model on a legacy version

import scipy.misc
import os
import pandas as pd
import pickle
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

keras = tf.compat.v1.keras
Sequence = keras.utils.Sequence

from PIL import Image
import glob
#from pathlib import Path

2023-06-09 20:30:46.842731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Input shape of our images and some empty lists that we will populate later

input_shape = (66, 200, 3)
actual_angle=[]
smooth_angle=0
test_ids=[]

In [4]:
# Parameters for datagen.py

params = {'dim': (66,200,3),
          'batch_size': 32,
          'shuffle': True}

In [5]:
#Path for the test images.  These should be in a seperate folder than the training and validation images

path_name="/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"

In [6]:
# This opens the key file and creates two lists, the angle (target) and the ids which is the file name without jpg

f= open("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/SelfDrivingCarNvidiadatakey58000Plus.txt")                                 #read steering angles from disk and preprocess
data = f.read()
data = data.split()
for i in data:                                      #if the node ends with ".jpg" ignore it. It's for collecting angles
    if i[-1]=='g':
 
       i=i[:-4]
       test_ids.append(int(i))
    else:
        actual_angle.append(float(i))    

In [7]:
#These are the actual angles for the training.  We won't use these for training but you can see the actual vs 
#predicted angles as the model trains

labels={}
for i in test_ids:
    labels[i]=float(actual_angle[i-58000])* scipy.pi / 180

In [8]:
# load the model from disk.  This is the model that we created in the other notebook

filename = '/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/models/finalized_model2.sav'
loaded_model = pickle.load(open(filename, 'rb'))



2023-06-09 20:30:52.302072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
#Image of Steering Wheel.  The idea is we have one image of the steering wheel and rotate it based on predicted angle

steering_wheel = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/SteeringWheelImage/steering_wheel_image.png") #read steering image
#resize it
steering_wheel=cv2.resize(steering_wheel, (270,210))
#get it's shape
rows,cols,level = steering_wheel.shape

In [10]:
#Same preprocessing done on training and validation images

def preProcessing(img):
  
  #crops image, the goal is to get the road and nothing else.  Will depend on camera placement
  img= img[120:235,:,:]

  #converts to an image that changes colors but helps the computer see lane lines
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)

  #blurs the image improving model performance
  img =cv2.GaussianBlur(img, (3,3),0)

  #resizes the image as this is what Nvidia trained on
  img=cv2.resize(img, (200, 66))

  #Normalizes image.  Changes color values from 0 to 255 to 0 to 1.  This is to ensure that no
  # large numbers become more important on the model
  img = img/255

  return img

In [11]:
#This code loads and read some images that were reserved for testing.  It then uses the 
#model we trained in the other notebook to predict a steering wheel angle.  

X=[]
y=[]


for i in test_ids:
    

    image = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"+str(i)+".jpg")   #read images from disk
    image=preProcessing(image)
    
    X = np.asarray(image)
    X = X.reshape(1,66,200,3)

    result = loaded_model.predict(X)*180.0/scipy.pi  

    print("Actual Angle= {} Predicted Angle= {}".format(str(actual_angle[i-58000]),str(result.item())))   # This prints actual vs predicted angles.  I think you will need to convert Actual to Radians if you use it

    #this section just for the smoother rotation of streeing wheel.
    smooth_angle += 0.2 * pow(abs((result - smooth_angle)), 2.0/3.0)*(result - smooth_angle)/abs(result-smooth_angle)
    M=cv2.getRotationMatrix2D((cols/2,rows/2),-smooth_angle.item(),1)

    dst= cv2.warpAffine(steering_wheel, M, (cols,rows))

    #this section saves the image of the steering wheel 
    img=Image.fromarray(dst,'RGB')
    img.save("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/StillImages/SteeringWheel"+str(i)+".jpg")

    

1/1 [==============================] - 0s 258ms/step
Actual Angle= -16.639999 Predicted Angle= -21.684097290039062
1/1 [==============================] - 0s 21ms/step
Actual Angle= -16.639999 Predicted Angle= -26.870460510253906
1/1 [==============================] - 0s 24ms/step
Actual Angle= -16.940001 Predicted Angle= -28.56008529663086
1/1 [==============================] - 0s 23ms/step
Actual Angle= -17.24 Predicted Angle= -26.072269439697266
1/1 [==============================] - 0s 21ms/step
Actual Angle= -18.450001 Predicted Angle= -9.246659278869629
1/1 [==============================] - 0s 23ms/step
Actual Angle= -19.16 Predicted Angle= -30.855323791503906
1/1 [==============================] - 0s 24ms/step
Actual Angle= -19.66 Predicted Angle= -29.962993621826172
1/1 [==============================] - 0s 21ms/step
Actual Angle= -19.969999 Predicted Angle= -20.715198516845703
1/1 [==============================] - 0s 21ms/step
Actual Angle= -20.17 Predicted Angle= -10.3144292

1/1 [==============================] - 0s 23ms/step
Actual Angle= 1.71 Predicted Angle= 7.790820598602295
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.51 Predicted Angle= 7.675535202026367
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.01 Predicted Angle= 7.471748352050781
1/1 [==============================] - 0s 28ms/step
Actual Angle= 0.91 Predicted Angle= 6.943634986877441
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.91 Predicted Angle= 8.894719123840332
1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.81 Predicted Angle= 7.87214994430542
1/1 [==============================] - 0s 25ms/step
Actual Angle= 0.91 Predicted Angle= 2.1643130779266357
1/1 [==============================] - 0s 32ms/step
Actual Angle= 1.11 Predicted Angle= 5.875629901885986
1/1 [==============================] - 0s 25ms/step
Actual Angle= 1.51 Predicted Angle= 5.7348246574401855
1/1 [==============================] - 0s 26m

1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.0 Predicted Angle= 1.9146164655685425
1/1 [==============================] - 0s 23ms/step
Actual Angle= -5.34 Predicted Angle= 0.7878814935684204
1/1 [==============================] - 0s 23ms/step
Actual Angle= -5.85 Predicted Angle= 0.8774001002311707
1/1 [==============================] - 0s 22ms/step
Actual Angle= -6.66 Predicted Angle= -0.1885533481836319
1/1 [==============================] - 0s 37ms/step
Actual Angle= -7.87 Predicted Angle= -5.540825843811035
1/1 [==============================] - 0s 28ms/step
Actual Angle= -8.87 Predicted Angle= -4.303462982177734
1/1 [==============================] - 0s 24ms/step
Actual Angle= -10.18 Predicted Angle= -7.881262302398682
1/1 [==============================] - 0s 24ms/step
Actual Angle= -11.39 Predicted Angle= -6.996765613555908
1/1 [==============================] - 0s 24ms/step
Actual Angle= -11.9 Predicted Angle= -5.686504364013672
1/1 [======================

1/1 [==============================] - 0s 24ms/step
Actual Angle= -1.51 Predicted Angle= 1.1346672773361206
1/1 [==============================] - 0s 24ms/step
Actual Angle= -1.11 Predicted Angle= -6.094519138336182
1/1 [==============================] - 0s 23ms/step
Actual Angle= -1.01 Predicted Angle= -1.013230323791504
1/1 [==============================] - 0s 23ms/step
Actual Angle= -0.2 Predicted Angle= -0.17652197182178497
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.61 Predicted Angle= -1.8950008153915405
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.91 Predicted Angle= 1.1968804597854614
1/1 [==============================] - 0s 25ms/step
Actual Angle= 0.0 Predicted Angle= 1.8003137111663818
1/1 [==============================] - 0s 28ms/step
Actual Angle= 1.21 Predicted Angle= 1.3029148578643799
1/1 [==============================] - 0s 89ms/step
Actual Angle= 1.21 Predicted Angle= -3.4467720985412598
1/1 [==========================

1/1 [==============================] - 0s 26ms/step
Actual Angle= 3.53 Predicted Angle= -5.663906097412109
1/1 [==============================] - 0s 24ms/step
Actual Angle= 3.03 Predicted Angle= -5.718972682952881
1/1 [==============================] - 0s 23ms/step
Actual Angle= 2.62 Predicted Angle= -5.37486457824707
1/1 [==============================] - 0s 22ms/step
Actual Angle= 2.12 Predicted Angle= -0.3861788809299469
1/1 [==============================] - 0s 24ms/step
Actual Angle= 1.31 Predicted Angle= -2.4918603897094727
1/1 [==============================] - 0s 22ms/step
Actual Angle= 0.91 Predicted Angle= -0.4387910068035126
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.71 Predicted Angle= -2.0949676036834717
1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.61 Predicted Angle= -6.061643600463867
1/1 [==============================] - 0s 36ms/step
Actual Angle= 0.61 Predicted Angle= -6.599677085876465
1/1 [=============================

...

1/1 [==============================] - 0s 78ms/step
Actual Angle= 0.0 Predicted Angle= 29.469200134277344
1/1 [==============================] - 0s 29ms/step
Actual Angle= 7.26 Predicted Angle= 24.291372299194336
1/1 [==============================] - 0s 24ms/step
Actual Angle= 7.26 Predicted Angle= 22.524044036865234
1/1 [==============================] - 0s 26ms/step
Actual Angle= 7.76 Predicted Angle= 14.825263023376465
1/1 [==============================] - 0s 26ms/step
Actual Angle= 8.17 Predicted Angle= -17.825918197631836
1/1 [==============================] - 0s 33ms/step
Actual Angle= 8.47 Predicted Angle= -5.468461990356445
1/1 [==============================] - 0s 31ms/step
Actual Angle= 8.97 Predicted Angle= -11.49280834197998
1/1 [==============================] - 0s 26ms/step
Actual Angle= 9.28 Predicted Angle= -24.6712646484375
1/1 [==============================] - 0s 30ms/step
Actual Angle= 9.68 Predicted Angle= -32.675533294677734
1/1 [==============================] 

1/1 [==============================] - 0s 26ms/step
Actual Angle= -148.940002 Predicted Angle= -37.05791473388672
1/1 [==============================] - 0s 25ms/step
Actual Angle= -150.050003 Predicted Angle= -33.90980529785156
1/1 [==============================] - 0s 23ms/step
Actual Angle= -151.059998 Predicted Angle= -35.296817779541016
1/1 [==============================] - 0s 26ms/step
Actual Angle= -152.169998 Predicted Angle= -34.46158218383789
1/1 [==============================] - 0s 24ms/step
Actual Angle= -152.570007 Predicted Angle= -40.02518081665039
1/1 [==============================] - 0s 25ms/step
Actual Angle= -152.770004 Predicted Angle= -33.60808563232422
1/1 [==============================] - 0s 26ms/step
Actual Angle= -152.869995 Predicted Angle= -42.068359375
1/1 [==============================] - 0s 23ms/step
Actual Angle= -152.869995 Predicted Angle= -55.86639404296875
1/1 [==============================] - 0s 24ms/step
Actual Angle= -152.869995 Predicted Angl

1/1 [==============================] - 0s 24ms/step
Actual Angle= 3.23 Predicted Angle= -6.131380558013916
1/1 [==============================] - 0s 25ms/step
Actual Angle= 3.23 Predicted Angle= -2.887143850326538
1/1 [==============================] - 0s 30ms/step
Actual Angle= 3.23 Predicted Angle= -3.3329334259033203
1/1 [==============================] - 0s 26ms/step
Actual Angle= 3.23 Predicted Angle= -7.536887168884277
1/1 [==============================] - 0s 25ms/step
Actual Angle= 3.23 Predicted Angle= -7.427940368652344
1/1 [==============================] - 0s 27ms/step
Actual Angle= 3.23 Predicted Angle= -9.090330123901367
1/1 [==============================] - 0s 22ms/step
Actual Angle= 3.23 Predicted Angle= -6.318168640136719
1/1 [==============================] - 0s 25ms/step
Actual Angle= 3.13 Predicted Angle= -8.82116413116455
1/1 [==============================] - 0s 32ms/step
Actual Angle= 3.03 Predicted Angle= -9.871954917907715
1/1 [==============================] 

1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.3 Predicted Angle= -4.958072662353516
1/1 [==============================] - 0s 22ms/step
Actual Angle= 0.3 Predicted Angle= -21.817691802978516
1/1 [==============================] - 0s 25ms/step
Actual Angle= 0.4 Predicted Angle= -30.0815372467041
1/1 [==============================] - 0s 25ms/step
Actual Angle= 0.4 Predicted Angle= 2.798307418823242
1/1 [==============================] - 0s 24ms/step
Actual Angle= 0.4 Predicted Angle= 3.0227694511413574
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.4 Predicted Angle= -1.4017040729522705
1/1 [==============================] - 0s 22ms/step
Actual Angle= 0.4 Predicted Angle= -5.895018100738525
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.4 Predicted Angle= -10.343523025512695
1/1 [==============================] - 0s 23ms/step
Actual Angle= 0.4 Predicted Angle= -7.564759731292725
1/1 [==============================] - 0s 24m

1/1 [==============================] - 0s 24ms/step
Actual Angle= -2.32 Predicted Angle= 0.8813487887382507
1/1 [==============================] - 0s 24ms/step
Actual Angle= -2.32 Predicted Angle= 0.8326811790466309
1/1 [==============================] - 0s 24ms/step
Actual Angle= -2.32 Predicted Angle= 1.1767979860305786
1/1 [==============================] - 0s 23ms/step
Actual Angle= -2.32 Predicted Angle= 1.476704716682434
1/1 [==============================] - 0s 26ms/step
Actual Angle= -2.32 Predicted Angle= 0.8587635159492493
1/1 [==============================] - 0s 23ms/step
Actual Angle= -2.32 Predicted Angle= 1.3415887355804443
1/1 [==============================] - 0s 24ms/step
Actual Angle= -2.32 Predicted Angle= 0.9101568460464478
1/1 [==============================] - 0s 29ms/step
Actual Angle= -2.32 Predicted Angle= 1.0292339324951172
1/1 [==============================] - 0s 26ms/step
Actual Angle= -2.32 Predicted Angle= 1.3677821159362793
1/1 [========================

In [26]:
#Makes a Video of the Steering wheel with predictions.  This stitches all the still images of the steering wheel
#rotated based on our predictions into a video and saves it

img_array = []
for i in test_ids:
    img = cv2.imread('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/StillImages/SteeringWheel'+str(i)+'.jpg')
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)


out = cv2.VideoWriter('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/steering_wheel.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 15, size) 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [15]:
# This takes the still images that we used for predictions and Makes a video of test images road driving

img_array = []
for i in test_ids:
    img = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages58000plus/"+str(i)+".jpg")
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    
out = cv2.VideoWriter('/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Videos/Dave2Nvidea6_28_22/PredictDrive.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
#At this point you have two videos, the steering wheel and the road.  I used Imovie on Mac to overlay the
#steering wheel video on top of the road video.  